In [1]:
import numpy as np

# ----------------------------
# Helpers: rotations & rays
# ----------------------------
def deg2rad(d): return d * np.pi / 180.0

def R_from_yaw_pitch_roll(yaw_deg, pitch_deg, roll_deg):
    """
    Returns R(world->camera) using ZYX (yaw, then pitch, then roll).
    yaw about +Z, pitch about +Y (nose down positive), roll about +X.
    """
    yaw, pitch, roll = map(deg2rad, (yaw_deg, pitch_deg, roll_deg))
    cz, sz = np.cos(yaw),   np.sin(yaw)
    cy, sy = np.cos(pitch), np.sin(pitch)
    cx, sx = np.cos(roll),  np.sin(roll)

    Rz = np.array([[cz, -sz, 0],
                   [sz,  cz, 0],
                   [ 0,   0, 1]])
    Ry = np.array([[ cy, 0, sy],
                   [  0, 1,  0],
                   [-sy, 0, cy]])
    Rx = np.array([[1,  0,   0],
                   [0, cx, -sx],
                   [0, sx,  cx]])
    # World -> Camera
    return Rx @ Ry @ Rz

def pixel_ray_world(u, v, K, R_world_to_cam):
    """
    u,v (pixels) -> unit ray direction in WORLD frame.
    K: intrinsics, R_world_to_cam: extrinsics.
    """
    # camera-frame (normalized) ray
    p = np.array([u, v, 1.0])
    d_cam = np.linalg.inv(K) @ p
    d_cam = d_cam / np.linalg.norm(d_cam)
    # rotate to world: d_world = R^T * d_cam  (since R maps world->cam)
    d_world = R_world_to_cam.T @ d_cam
    return d_world / np.linalg.norm(d_world)

def triangulate_two_rays(C1, d1, C2, d2):
    """
    Two skew lines: L1 = C1 + λ1 d1, L2 = C2 + λ2 d2 (directions unit).
    Returns midpoint P of shortest segment between the lines and residual length.
    """
    d1 = d1 / np.linalg.norm(d1)
    d2 = d2 / np.linalg.norm(d2)

    a = np.dot(d1, d1)
    b = np.dot(d1, d2)
    c = np.dot(d2, d2)
    r = C2 - C1
    e = np.dot(d1, r)
    f = np.dot(d2, r)

    M = np.array([[a, -b],
                  [-b, c]])
    rhs = np.array([e, f])
    lam1, lam2 = np.linalg.solve(M, rhs)

    P1 = C1 + lam1 * d1
    P2 = C2 + lam2 * d2
    P  = 0.5 * (P1 + P2)
    residual = np.linalg.norm(P1 - P2)
    return P, residual, (lam1, lam2), (P1, P2)

def intersect_ground_from_point_and_dir(P, d_world):
    """
    Ray starting at P going along d_world; intersect Z=0 plane.
    """
    if abs(d_world[2]) < 1e-9:
        return None  # parallel to ground
    lam = -P[2] / d_world[2]
    return P + lam * d_world

def bearing_deg_from_tower(tower_xy, target_xy):
    """
    Bearing (degrees) from tower to target in ENU (X=East, Y=North).
    0° = North, 90° = East.
    """
    dx = target_xy[0] - tower_xy[0]
    dy = target_xy[1] - tower_xy[1]
    brad = np.arctan2(dx, dy)  # swap to make 0°=North, increasing clockwise
    bdeg = (brad * 180/np.pi) % 360.0
    dist = np.hypot(dx, dy)
    return bdeg, dist

# ----------------------------
# Sample Scene Setup (EDIT ME)
# ----------------------------

# Image size (just for intuition)
W, H = 1920, 1080

# Intrinsics (fx=fy=1200 px, principal point at image center)
fx = fy = 1200.0
cx = W/2
cy = H/2
K = np.array([[fx,   0, cx],
              [ 0, fy, cy],
              [ 0,   0,  1]], dtype=float)

# Tower positions in a local ENU world frame (meters)
# Put Tower-1 at origin on ground; add camera heights.
tower1_base = np.array([   0.0,    0.0, 0.0])
tower2_base = np.array([ 300.0,   50.0, 0.0])

cam1_height = 22.0  # meters above ground
cam2_height = 24.0

C1 = tower1_base + np.array([0,0,cam1_height])  # camera center world (tower-1)
C2 = tower2_base + np.array([0,0,cam2_height])  # camera center world (tower-2)

# Camera orientations (world->camera) — tweak as needed.
# Example: Tower-1 looks roughly East (yaw=0), pitched down 10°
#          Tower-2 looks roughly West (yaw=180), pitched down 8°
R1 = R_from_yaw_pitch_roll(yaw_deg=0.0,   pitch_deg=-10.0, roll_deg=0.0)
R2 = R_from_yaw_pitch_roll(yaw_deg=180.0, pitch_deg=-8.0,  roll_deg=0.0)

# Detected fire centroids in pixels (u,v) in each image (sample numbers)
# (Right half of cam1; left half of cam2)
u1, v1 = 1300.0, 700.0
u2, v2 =  500.0, 680.0

# ----------------------------
# Back-project rays & triangulate
# ----------------------------
d1 = pixel_ray_world(u1, v1, K, R1)
d2 = pixel_ray_world(u2, v2, K, R2)

P, residual, (lam1, lam2), (P1, P2) = triangulate_two_rays(C1, d1, C2, d2)

print("Camera-1 center C1:", C1)
print("Camera-2 center C2:", C2)
print("\nRay directions (world):")
print("d1:", d1)
print("d2:", d2)

print("\nTriangulated fire point P (X,Y,Z) [m]:", P)
print("Closest-approach residual between rays [m]:", residual)
print("Ray parameters λ1, λ2:", lam1, lam2)

# ----------------------------
# Optional: project average ray to ground (Z=0)
# ----------------------------
# Use average direction from the two ray points near P:
avg_dir = (P2 - P1)
if np.linalg.norm(avg_dir) > 1e-9:
    avg_dir = avg_dir / np.linalg.norm(avg_dir)
else:
    avg_dir = d1  # fallback

ground_pt = intersect_ground_from_point_and_dir(P, avg_dir)
if ground_pt is not None:
    print("\nGround intersection (Z=0) [m]:", ground_pt)
    bdeg, dist = bearing_deg_from_tower(tower1_base[:2], ground_pt[:2])
    print(f"Bearing from Tower-1: {bdeg:.1f}°, Range: {dist:.1f} m")
else:
    print("\nRay is parallel to ground—no Z=0 intersection.")


Camera-1 center C1: [ 0.  0. 22.]
Camera-2 center C2: [300.  50.  24.]

Ray directions (world):
d1: [0.4319926  0.12724086 0.89285618]
d2: [ 0.2231799  -0.10829635  0.96874281]

Triangulated fire point P (X,Y,Z) [m]: [ 780.73981056   45.42595425 1808.79639709]
Closest-approach residual between rays [m]: 432.85779480739376
Ray parameters λ1, λ2: 1938.4060632737603 1900.2721299128484

Ground intersection (Z=0) [m]: [2607.55705022 6535.66448029    0.        ]
Bearing from Tower-1: 21.8°, Range: 7036.6 m
